In [1]:
import os
import pandas as pd

In [7]:
df = pd.read_excel('C:/Users/User/Downloads/Telegram Desktop/ТСГ_6_Бобринець_05_2016_озимий_ячмінь.xls')
df.head(50)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,ТСГ - 6,NaN,NaN,NaN,NaN,NaN
1,ГІДРОМЕТЕОРОЛОГІЧНА СЛУЖБА УКРАЇНИ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Вологість у відсотках від маси абсолютно сухог...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Станція Бобринець ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Культура,NaN,NaN,озимий ячмінь,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Ґрунт,NaN,чорнозем зв.середньогумусний важкосугл.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_BD = pd.read_excel('C:/Users/User/Meteo_DB/DATAbase_soil water/Запаси вологи в ґрунті.xlsx')
df_BD.columns

Index(['Рік', 'Область', 'Метеостанція', 'Місяць', 'Дата', 'Культура ',
       'Попередня культура', 'Тип ґрунту', 'Ділянка',
       'Об'ємна маса ґрунту на глибині ґрунту 10 см, г/ см³',
       'Об'ємна маса ґрунту на глибині ґрунту 20 см, г/ см³',
       'Об'ємна маса ґрунту на глибині ґрунту 30 см, г/ см³',
       'Об'ємна маса ґрунту на глибині ґрунту 40 см, г/ см³',
       'Об'ємна маса ґрунту на глибині ґрунту 50 см, г/ см³',
       'Об'ємна маса ґрунту на глибині ґрунту 60 см, г/ см³',
       'Об'ємна маса ґрунту на глибині ґрунту 70 см, г/ см³',
       'Об'ємна маса ґрунту на глибині ґрунту 80 см, г/ см³',
       'Об'ємна маса ґрунту на глибині ґрунту 90 см, г/ см³',
       'Об'ємна маса ґрунту на глибині ґрунту 100 см, г/ см³',
       'Запаси непродуктивної вологи на глибині ґрунту 10 см, мм',
       'Запаси непродуктивної вологи на глибині ґрунту 20 см, мм',
       'Запаси непродуктивної вологи на глибині ґрунту 30 см, мм',
       'Запаси непродуктивної вологи на глибині ґрун

In [2]:
def extract_data_from_excel(file_path):
    # Load all sheets from the Excel file
    xls = pd.ExcelFile(file_path)
    df_final = pd.DataFrame()

    # Iterate through each sheet in the Excel file
    for sheet_name in xls.sheet_names:
        xls_data = pd.read_excel(xls, sheet_name=sheet_name)  # Load the current sheet

        # Initialize an empty DataFrame for storing the results for this sheet
        df = pd.DataFrame()

        # Iterate through each cell in the dataframe
        data_index = 0
        for idx, row in xls_data.iterrows():
            for col_idx, cell in enumerate(row):
                if pd.notna(cell):
                    previous_cell = xls_data.iloc[idx-1, col_idx] if idx > 0 else None  # Cell above the current cell
                    previous_3rd_cell = xls_data.iloc[idx-3, col_idx] if idx > 2 else None  # Three cells above the current cell

                    # Condition for date and "середня" related data
                    if "середня" in str(cell) and ("4" in str(previous_cell) or "2" in str(previous_3rd_cell)):
                        date_slice = xls_data.iloc[max(0, idx-3):idx+1, 0].dropna()  # Use max to avoid negative indexing issues
                        date = date_slice.iloc[-1] if not date_slice.empty else None

                        if date is not None:
                            data_index = idx  # Use this index for data storage for related conditions
                            df.loc[data_index, 'Дата'] = date  # Store the date for this condition
                        
                    # Oblast information
                    if "Область" in str(cell):
                        oblast_info = xls_data.iloc[idx, col_idx+1:col_idx+10].dropna().tolist()
                        df.loc[0, 'Область'] = ' '.join(oblast_info)
                    # Meteosation information
                    if "Станція" in str(cell):
                        station_info = xls_data.iloc[idx, col_idx+1:col_idx+8].dropna().tolist()
                        if station_info == []:
                            df.loc[0, 'Метеостанція'] = cell
                        else:
                            df.loc[0, 'Метеостанція'] = ' '.join(station_info) 
                    # Culture and Soil information
                    if "Культура" in str(cell):
                        culture_info = xls_data.iloc[idx, col_idx+1:col_idx+8].dropna().tolist()
                        df.loc[0, 'Культура'] = ' '.join(culture_info)
                    if "Ґрунт" in str(cell):
                        soil_info = xls_data.iloc[idx, col_idx+1:col_idx+8].dropna().tolist()
                        df.loc[0, 'Ґрунт'] = ' '.join(soil_info)
                    # Plot information
                    if "Ділянка" in str(cell):
                        #plot_info = xls_data.iloc[idx, col_idx+1:col_idx+8].dropna().tolist()
                        plot_info = [str(item) for item in xls_data.iloc[idx, col_idx+1:col_idx+8].dropna().tolist()]
                        if plot_info == []:
                            df.loc[0, 'Ділянка'] = cell
                        else:
                            df.loc[0, 'Ділянка'] = ' '.join(plot_info)                                                
                        

                    # Handling various conditions for moisture and density
                    conditions = {
                        "Об'ємна маса": "Об'ємна маса ґрунту на глибині ґрунту {} см, г/см3",
                        "непродуктивної": "Запаси непродуктивної вологи на глибині ґрунту {} см, мм",
                        "продуктивної при НВ": "Запаси продуктивної вологи при НВ на глибині ґрунту {} см, мм"
                    }

                    for key, format_str in conditions.items():
                        if key in str(cell):
                            data = xls_data.iloc[idx, col_idx+1:col_idx+101].dropna().tolist()[:10]
                            columns = [format_str.format(i*10) for i in range(1, len(data)+1)]
                            df.loc[data_index, columns] = data

                    # Additional moisture content conditions
                    if "середня" in str(cell) and ("4" in str(previous_cell) or "2" in str(previous_3rd_cell)):
                        average_moisture = xls_data.iloc[idx, col_idx+1:col_idx+101].dropna().tolist()[:10]  #Use idx+101 to ensure covering of empty and cells with values, then [:10] took only 10 cells with values
                        average_columns = [f'Вологість від абсолютно сухого ґрунту (середня) на глибині ґрунту {i*10} см, %' for i in range(1, len(average_moisture)+1)]
                        df.loc[data_index, average_columns] = average_moisture

                    # Moisture by layers
                    if "по шарах" in str(cell):
                        if "середня" in str(previous_cell):
                            total_moisture_layers = xls_data.iloc[idx, col_idx+1:col_idx+101].dropna().tolist()[:10]
                            total_layers_columns = [f'Запаси загальної вологи по шарах на глибині ґрунту {i*10} см, мм' for i in range(1, len(total_moisture_layers)+1)]
                            df.loc[data_index, total_layers_columns] = total_moisture_layers
                        elif "по шарах" in str(previous_cell):
                            productive_moisture_layers = xls_data.iloc[idx, col_idx+1:col_idx+101].dropna().tolist()[:10]
                            productive_layers_columns = [f'Запаси продуктивної вологи по шарах на глибині ґрунту {i*10} см, мм' for i in range(1, len(productive_moisture_layers)+1)]
                            df.loc[data_index, productive_layers_columns] = productive_moisture_layers

                    if "наростаючим" in str(cell) and "по шарах" in str(previous_cell):
                        cumulative_moisture = xls_data.iloc[idx, col_idx+1:col_idx+101].dropna().tolist()[:10]
                        cumulative_columns = [f'Запаси продуктивної вологи наростаючим підсумком на глибині ґрунту {i*10} см, мм' for i in range(1, len(cumulative_moisture)+1)]
                        df.loc[data_index, cumulative_columns] = cumulative_moisture

        # Append the DataFrame of the current sheet to the final DataFrame
        df_final = pd.concat([df_final, df], ignore_index=True)

    return df_final

In [4]:
path_all_folders = 'C:/Users/User/Meteo_DB/DATAbase_soil water'

#Create and empty database
df_DB = pd.DataFrame(columns = ['file_path', 'Рік','Область', 'Метеостанція', 'Ділянка', 'Культура', 'Ґрунт', 'Дата',
       "Об'ємна маса ґрунту на глибині ґрунту 10 см, г/см3",
       "Об'ємна маса ґрунту на глибині ґрунту 20 см, г/см3",
       "Об'ємна маса ґрунту на глибині ґрунту 30 см, г/см3",
       "Об'ємна маса ґрунту на глибині ґрунту 40 см, г/см3",
       "Об'ємна маса ґрунту на глибині ґрунту 50 см, г/см3",
       "Об'ємна маса ґрунту на глибині ґрунту 60 см, г/см3",
       "Об'ємна маса ґрунту на глибині ґрунту 70 см, г/см3",
       "Об'ємна маса ґрунту на глибині ґрунту 80 см, г/см3",
       "Об'ємна маса ґрунту на глибині ґрунту 90 см, г/см3",
       "Об'ємна маса ґрунту на глибині ґрунту 100 см, г/см3",
       'Запаси непродуктивної вологи на глибині ґрунту 10 см, мм',
       'Запаси непродуктивної вологи на глибині ґрунту 20 см, мм',
       'Запаси непродуктивної вологи на глибині ґрунту 30 см, мм',
       'Запаси непродуктивної вологи на глибині ґрунту 40 см, мм',
       'Запаси непродуктивної вологи на глибині ґрунту 50 см, мм',
       'Запаси непродуктивної вологи на глибині ґрунту 60 см, мм',
       'Запаси непродуктивної вологи на глибині ґрунту 70 см, мм',
       'Запаси непродуктивної вологи на глибині ґрунту 80 см, мм',
       'Запаси непродуктивної вологи на глибині ґрунту 90 см, мм',
       'Запаси непродуктивної вологи на глибині ґрунту 100 см, мм',
       'Запаси продуктивної вологи при НВ на глибині ґрунту 10 см, мм',
       'Запаси продуктивної вологи при НВ на глибині ґрунту 20 см, мм',
       'Запаси продуктивної вологи при НВ на глибині ґрунту 30 см, мм',
       'Запаси продуктивної вологи при НВ на глибині ґрунту 40 см, мм',
       'Запаси продуктивної вологи при НВ на глибині ґрунту 50 см, мм',
       'Запаси продуктивної вологи при НВ на глибині ґрунту 60 см, мм',
       'Запаси продуктивної вологи при НВ на глибині ґрунту 70 см, мм',
       'Запаси продуктивної вологи при НВ на глибині ґрунту 80 см, мм',
       'Запаси продуктивної вологи при НВ на глибині ґрунту 90 см, мм',
       'Запаси продуктивної вологи при НВ на глибині ґрунту 100 см, мм',
       'Вологість від абсолютно сухого ґрунту (середня) на глибині ґрунту 10 см, %',
       'Вологість від абсолютно сухого ґрунту (середня) на глибині ґрунту 20 см, %',
       'Вологість від абсолютно сухого ґрунту (середня) на глибині ґрунту 30 см, %',
       'Вологість від абсолютно сухого ґрунту (середня) на глибині ґрунту 40 см, %',
       'Вологість від абсолютно сухого ґрунту (середня) на глибині ґрунту 50 см, %',
       'Вологість від абсолютно сухого ґрунту (середня) на глибині ґрунту 60 см, %',
       'Вологість від абсолютно сухого ґрунту (середня) на глибині ґрунту 70 см, %',
       'Вологість від абсолютно сухого ґрунту (середня) на глибині ґрунту 80 см, %',
       'Вологість від абсолютно сухого ґрунту (середня) на глибині ґрунту 90 см, %',
       'Вологість від абсолютно сухого ґрунту (середня) на глибині ґрунту 100 см, %',
       'Запаси загальної вологи по шарах на глибині ґрунту 10 см, мм',
       'Запаси загальної вологи по шарах на глибині ґрунту 20 см, мм',
       'Запаси загальної вологи по шарах на глибині ґрунту 30 см, мм',
       'Запаси загальної вологи по шарах на глибині ґрунту 40 см, мм',
       'Запаси загальної вологи по шарах на глибині ґрунту 50 см, мм',
       'Запаси загальної вологи по шарах на глибині ґрунту 60 см, мм',
       'Запаси загальної вологи по шарах на глибині ґрунту 70 см, мм',
       'Запаси загальної вологи по шарах на глибині ґрунту 80 см, мм',
       'Запаси загальної вологи по шарах на глибині ґрунту 90 см, мм',
       'Запаси загальної вологи по шарах на глибині ґрунту 100 см, мм',
       'Запаси продуктивної вологи по шарах на глибині ґрунту 10 см, мм',
       'Запаси продуктивної вологи по шарах на глибині ґрунту 20 см, мм',
       'Запаси продуктивної вологи по шарах на глибині ґрунту 30 см, мм',
       'Запаси продуктивної вологи по шарах на глибині ґрунту 40 см, мм',
       'Запаси продуктивної вологи по шарах на глибині ґрунту 50 см, мм',
       'Запаси продуктивної вологи по шарах на глибині ґрунту 60 см, мм',
       'Запаси продуктивної вологи по шарах на глибині ґрунту 70 см, мм',
       'Запаси продуктивної вологи по шарах на глибині ґрунту 80 см, мм',
       'Запаси продуктивної вологи по шарах на глибині ґрунту 90 см, мм',
       'Запаси продуктивної вологи по шарах на глибині ґрунту 100 см, мм',
       'Запаси продуктивної вологи наростаючим підсумком на глибині ґрунту 10 см, мм',
       'Запаси продуктивної вологи наростаючим підсумком на глибині ґрунту 20 см, мм',
       'Запаси продуктивної вологи наростаючим підсумком на глибині ґрунту 30 см, мм',
       'Запаси продуктивної вологи наростаючим підсумком на глибині ґрунту 40 см, мм',
       'Запаси продуктивної вологи наростаючим підсумком на глибині ґрунту 50 см, мм',
       'Запаси продуктивної вологи наростаючим підсумком на глибині ґрунту 60 см, мм',
       'Запаси продуктивної вологи наростаючим підсумком на глибині ґрунту 70 см, мм',
       'Запаси продуктивної вологи наростаючим підсумком на глибині ґрунту 80 см, мм',
       'Запаси продуктивної вологи наростаючим підсумком на глибині ґрунту 90 см, мм',
       'Запаси продуктивної вологи наростаючим підсумком на глибині ґрунту 100 см, мм'])

for year in ['2016', '2017', '2018', '2019', '2020', '2021']:
    print(year)
    for folder_obl in os.listdir(f"{path_all_folders}/ТСГ-6_{year}"):
        print(folder_obl)
        for folder_st in os.listdir(f"{path_all_folders}/ТСГ-6_{year}/{folder_obl}"):
            for folder_crop in os.listdir(f"{path_all_folders}/ТСГ-6_{year}/{folder_obl}/{folder_st}"):
                for file in os.listdir(f"{path_all_folders}/ТСГ-6_{year}/{folder_obl}/{folder_st}/{folder_crop}"):
                    file_path = f'{path_all_folders}/ТСГ-6_{year}/{folder_obl}/{folder_st}/{folder_crop}/{file}'
                    df = extract_data_from_excel(file_path)
                    df.loc[0, 'file_path'] = f'ТСГ-6_{year}/{folder_obl}/{folder_st}/{folder_crop}/{file}'
                    df.loc[0, 'Рік'] = year
                    df_DB = pd.concat([df_DB, df], ignore_index=True)
#df_DB.to_excel('test.xlsx')
df_DB.to_excel('DB_Soil_water_2016_2021_raw_2.xlsx')

2016
ТСГ-6_2016_Івано-Франківська область
ТСГ-6_2016_Вінницька область
ТСГ-6_2016_Волинська область
ТСГ-6_2016_Дніпропетровська область
ТСГ-6_2016_Донецький РЦГМ
ТСГ-6_2016_Житомирськая область
ТСГ-6_2016_Закарпатський ЦГМ
ТСГ-6_2016_Запорізька область
ТСГ-6_2016_Кіровоградська область
ТСГ-6_2016_Київська область
ТСГ-6_2016_Луганськая область
ТСГ-6_2016_Львівська область
ТСГ-6_2016_Миколаївська область
ТСГ-6_2016_Одеська область
ТСГ-6_2016_Полтавська область
ТСГ-6_2016_Рівненська область
ТСГ-6_2016_Сумський  ЦГМ
ТСГ-6_2016_Тернопільська область
ТСГ-6_2016_Харкiвська область
ТСГ-6_2016_Херсонська область
ТСГ-6_2016_Хмельницький обласний центр
ТСГ-6_2016_Черкасськая область
ТСГ-6_2016_Чернівецька область
ТСГ-6_2016_Чернігівськая область
2017
ТСГ-6_2017_Івано-Франківська область
ТСГ-6_2017_Вінницька область
ТСГ-6_2017_Волинська область
ТСГ-6_2017_Дніпропетровська область
ТСГ-6_2017_Донецький РЦГМ
ТСГ-6_2017_Житомирська область
ТСГ-6_2017_Закарпатський ЦГМ
ТСГ-6_2017_Запоріжська область
ТС

In [146]:
#Check folder structure and calculate the number of files
path_all_folders = 'C:/Users/User/Meteo_DB/DATAbase_soil water'
file_count = 0  # Initialize a counter for the number of files

# Check folder structure
for year in ['2016', '2017', '2018', '2019', '2020', '2021']:
    year_path = f"{path_all_folders}/ТСГ-6_{year}"
    if os.path.exists(year_path):  # Check if the year directory exists
        for folder_obl in os.listdir(year_path):
            obl_path = os.path.join(year_path, folder_obl)
            if os.path.exists(obl_path):  # Check if the obl directory exists
                for folder_st in os.listdir(obl_path):
                    st_path = os.path.join(obl_path, folder_st)
                    if os.path.exists(st_path):  # Check if the st directory exists
                        for folder_crop in os.listdir(st_path):
                            crop_path = os.path.join(st_path, folder_crop)
                            if os.path.exists(crop_path):  # Check if the crop directory exists
                                local_count = 0  # Counter for files within the current crop folder
                                for file in os.listdir(crop_path):
                                    file_path = os.path.join(crop_path, file)
                                    if os.path.isfile(file_path):  # Check if it is indeed a file
                                        file_count += 1
                                        local_count += 1
                                    else:
                                        print(f"Not a file or missing: {file_path}")
                                print(f"Total files in {crop_path}: {local_count}")
    else:
        print(f"Directory does not exist: {year_path}")

print(f"Total number of files found: {file_count}")

Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Івано-Франківська область\Долина\зяб: 3
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Івано-Франківська область\Долина\картопля: 4
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Івано-Франківська область\Долина\озима пшениця: 7
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Івано-Франківська область\Коломия\зяб: 3
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Івано-Франківська область\Коломия\картопля: 4
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Івано-Франківська область\Коломия\озима пшениця: 6
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Івано-Франківська область\Коломия\соя: 6
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Івано-Франківська область\Коломия\ярий ячмінь:

Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Кіровоградська область\Знам'янка\Озима пшениця: 11
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Кіровоградська область\Знам'янка\Озимий ріпак: 9
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Кіровоградська область\Знам'янка\Соя: 6
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Кіровоградська область\Новомиргород\Зяб: 3
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Кіровоградська область\Новомиргород\Кукурудза: 5
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Кіровоградська область\Новомиргород\Озима пшениця: 11
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Кіровоградська область\Новомиргород\Озимий ріпак: 10
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Кіровоградська област

Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Сумський  ЦГМ\Дружба\тритикале: 2
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Сумський  ЦГМ\Конотоп\гречка: 6
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Сумський  ЦГМ\Конотоп\картопля: 5
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Сумський  ЦГМ\Конотоп\кукурудза: 8
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Сумський  ЦГМ\Конотоп\овес: 5
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Сумський  ЦГМ\Конотоп\озима пшениця: 10
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Сумський  ЦГМ\Лебедин\горох: 3
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Сумський  ЦГМ\Лебедин\гречка: 5
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2016\ТСГ-6_2016_Сумський  ЦГ

Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2017\ТСГ-6_2017_Вінницька область\Гайсин\Ярий ячмінь: 5
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2017\ТСГ-6_2017_Вінницька область\Жмеринка\Картопля: 5
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2017\ТСГ-6_2017_Вінницька область\Жмеринка\Кукурудза: 2
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2017\ТСГ-6_2017_Вінницька область\Жмеринка\Озима пшениця: 9
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2017\ТСГ-6_2017_Вінницька область\Жмеринка\Соя: 4
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2017\ТСГ-6_2017_Вінницька область\Жмеринка\Ярий ячмінь: 6
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2017\ТСГ-6_2017_Вінницька область\Крижопіль\зяб: 1
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2017\ТСГ-6_2017_Вінницька область\Крижопіль\Кукурудза: 6
Total files in C:/Users/User/Meteo_DB/DATAbas

Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2017\ТСГ-6_2017_Луганська область\ТСХ-6_2017_М Новопсков\Озимий ячмінь: 4
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2017\ТСГ-6_2017_Луганська область\ТСХ-6_2017_М Новопсков\Соняшник: 3
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2017\ТСГ-6_2017_Луганська область\ТСХ-6_2017_М Новопсков\Ячмінь: 3
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2017\ТСГ-6_2017_Львівська область\Броди\картопля: 4
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2017\ТСГ-6_2017_Львівська область\Броди\кукурудза: 5
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2017\ТСГ-6_2017_Львівська область\Броди\озима пшениця: 9
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2017\ТСГ-6_2017_Львівська область\Кам'янка-Бузька\картопля: 5
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2017\ТСГ-6_2017_Львівська область\Кам'янка-Бузька\овес

...

Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2020\ТСГ-6_2019-2020_Житомирська\Овруч\Зяб: 3
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2020\ТСГ-6_2019-2020_Житомирська\Овруч\Картопля: 4
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2020\ТСГ-6_2019-2020_Житомирська\Овруч\Кукурудза: 5
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2020\ТСГ-6_2019-2020_Житомирська\Овруч\Овес: 5
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2020\ТСГ-6_2019-2020_Житомирська\Овруч\Озима пшениця: 7
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2020\ТСГ-6_2019-2020_Житомирська\Олевськ\Зяб: 4
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2020\ТСГ-6_2019-2020_Житомирська\Олевськ\Картопля: 4
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2020\ТСГ-6_2019-2020_Житомирська\Олевськ\Овес: 5
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2020\ТСГ-6_2019-2020_Ж

Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2020\ТСГ-6_2019-2020_Полтавська область\Гадяч\Озима пшениця: 7
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2020\ТСГ-6_2019-2020_Полтавська область\Гадяч\Соняшник: 6
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2020\ТСГ-6_2019-2020_Полтавська область\Гадяч\Ярий ячмінь: 4
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2020\ТСГ-6_2019-2020_Полтавська область\Кобеляки\Зяб: 2
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2020\ТСГ-6_2019-2020_Полтавська область\Кобеляки\Кукурудза: 6
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2020\ТСГ-6_2019-2020_Полтавська область\Кобеляки\Озима пшениця: 8
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2020\ТСГ-6_2019-2020_Полтавська область\Кобеляки\Соняшник: 6
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2020\ТСГ-6_2019-2020_Полтавська область\Лубни\Зяб: 2
Total fi

Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Вінницька область\Вінниця\Озима пшениця: 8
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Вінницька область\Вінниця\Соя: 5
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Вінницька область\Вінниця\Ярий ячмінь: 4
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Вінницька область\Гайсин\Зяб: 1
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Вінницька область\Гайсин\Картопля: 4
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Вінницька область\Гайсин\Озима пшениця: 7
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Вінницька область\Гайсин\Цукровий буряк: 5
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Вінницька область\Жмеринка\Зяб: 1
Total files in 

Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Львівська область\ТСГ-6_2021_Карпатська ГМО\Овес: 4
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Львівська область\ТСГ-6_2021_Карпатська ГМО\Озима пшениця: 7
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Львівська область\ТСГ-6_2021_Рава-Руська\картопля: 5
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Львівська область\ТСГ-6_2021_Рава-Руська\овес: 4
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Львівська область\ТСГ-6_2021_Рава-Руська\озима пшениця: 7
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Львівська область\ТСГ-6_2021_Яворів\зяб: 2
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Львівська область\ТСГ-6_2021_Яворів\картопля: 4
Total files in C:/Users/User/Meteo_DB/DAT

Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Херсонська область\Каховська ГМО\озимий ріпак: 8
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Херсонська область\Каховська ГМО\соняшник: 8
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Херсонська область\Н.Сірогози\горох: 4
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Херсонська область\Н.Сірогози\Зяб: 2
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Херсонська область\Н.Сірогози\озима пшениця: 8
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Херсонська область\Н.Сірогози\озимий ячмінь: 6
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Херсонська область\Н.Сірогози\соняшник: 5
Total files in C:/Users/User/Meteo_DB/DATAbase_soil water/ТСГ-6_2021\ТСГ-6_2020-2021_Херсонська обл